# QUANTUM PHASE ESTIMATION

## IMPORTS and SETUP

In [1]:
# general imports
import numpy as np
import math
import matplotlib.pyplot as plt
# magic word for producing visualizations in notebook
%matplotlib inline

# AWS imports: Import Amazon Braket SDK modules
from braket.circuits import Circuit, circuit
from braket.devices import LocalSimulator
from braket.aws import AwsDevice

# local imports
from utils_qpe import *

%load_ext autoreload
%autoreload 2

In [2]:
# set up device: local simulator or the on-demand simulator
device = LocalSimulator()
#device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")

### Pauli Matrices:
In some of our examples, we choose the unitary $U$ to be given by the **Pauli Matrices**, which we thus define as follows:

In [3]:
# Define Pauli matrices
Id = np.eye(2)             # Identity matrix
X = np.array([[0., 1.],
              [1., 0.]])   # Pauli X
Y = np.array([[0., -1.j],
              [1.j, 0.]])  # Pauli Y
Z = np.array([[1., 0.],
              [0., -1.]])  # Pauli Z

In [4]:
# set total number of qubits
precision_qubits = range(3)
query_qubits = [3]

# prepare query register
my_qpe_circ = Circuit().h(query_qubits)

# set unitary
unitary = X

# show small QPE example circuit
my_qpe_circ = my_qpe_circ.qpe(precision_qubits, query_qubits, unitary)
#print('QPE CIRCUIT:')
#print(my_qpe_circ)

In [5]:
out = run_qpe(
    unitary,
    precision_qubits,
    query_qubits,
    my_qpe_circ,
    device,
    items_to_keep=1,
    shots=1000)

In [6]:
postprocess_qpe_results(out)

Measurement counts: Counter({'0000': 523, '0001': 477})
Results in precision register: {'000': 1000}
QPE phase estimates: [0.0]
QPE eigenvalue estimates: [1.+0.j]


# TEST 

In this example, we choose the unitary to be a _random_ two-qubit unitary, diagonal in the computational basis. We initialize the query register to be in the eigenstate $|11\rangle$ of $U$, which we can prepare using that $|11\rangle = X\otimes X|00\rangle$.
In this case we should be able to read off the eigenvalue and phase from $U$ and verify that QPE gives the right answer.

In [7]:
# Generate a random 2 qubit unitary matrix:
from scipy.stats import unitary_group

# Fix random seed for reproducibility
np.random.seed(seed=46)

# Get random two-qubit unitary
random_unitary = unitary_group.rvs(2**2)

# Let's diagonalize this
evals = np.linalg.eig(random_unitary)[0]

# Since we want to be able to read off the eigenvalues of the unitary in question
# let's choose our unitary to be diagonal in this basis
unitary = np.diag(evals)

# Check that this is indeed unitary, and print it out:
print('Two-qubit random unitary:\n', np.round(unitary, 3))
print('Check for unitarity: ', np.allclose(np.eye(len(unitary)), unitary.dot(unitary.T.conj())))

# Print eigenvalues
print('Eigenvalues:', np.round(evals, 3))

Two-qubit random unitary:
 [[ 0.865-0.502j  0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j    -0.345-0.939j  0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j    -0.001+1.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.   +0.j    -0.836+0.549j]]
Check for unitarity:  True
Eigenvalues: [ 0.865-0.502j -0.345-0.939j -0.001+1.j    -0.836+0.549j]


Autovalore che ci aspettiamo di ricevere, considerando una precisione data $0.001$ con un errore $\epsilon = 5\% $:

In [8]:
numero_dcimali_stima = 3
max_error = 0.05
n_binary_estimation = -int(np.floor(np.log2(10**-numero_dcimali_stima)))


In [9]:
target_eigenvalue = evals[-1]
print('Target eigenvalue:', np.round(target_eigenvalue, numero_dcimali_stima))
target_phase = np.angle(target_eigenvalue,deg=False)/(2*np.pi)
print('Target phase:', np.round(target_phase, numero_dcimali_stima))

Target eigenvalue: (-0.836+0.549j)
Target phase: 0.408


In [11]:
# Set total number of precision qubits
number_precision_qubits = get_number_precision_qubits(n_binary_estimation,max_error)

# Define the set of precision qubits
precision_qubits = range(number_precision_qubits)

# Define the query qubits. We'll have them start after the precision qubits
query_qubits = [number_precision_qubits, number_precision_qubits+1]

# State preparation for eigenstate |1,1> of diagonal U
query = Circuit().x(query_qubits[0]).x(query_qubits[1])

# Run the test with U=X
out = run_qpe(unitary, precision_qubits, query_qubits, query, device)

postprocess_qpe_results(out)
# Postprocess results

eigenvalues = out['eigenvalues']
print('QPE eigenvalue estimates:', np.round(eigenvalues, 5))

# compare output to exact target values
print('Target eigenvalue:', np.round(evals[-1], 5))

Measurement counts: Counter({'0110100001011011': 741, '0110100001010111': 125, '0110100001011111': 43, '0110100001010011': 23, '0110100001100011': 16, '0110100001001111': 8, '0110100001100111': 6, '0110100001001011': 5, '0110100000110111': 3, '0110100001000111': 3, '0110100001101011': 3, '0110100001110011': 2, '0110100010000011': 2, '0110100001110111': 1, '0110100011001011': 1, '0110100000001011': 1, '0110100000011111': 1, '0110100010011111': 1, '0110011111111111': 1, '0110100010001011': 1, '0110100011000111': 1, '0110100000111111': 1, '0110100001000011': 1, '0110100101001011': 1, '0110100001111011': 1, '0110100001101111': 1, '1110110011101011': 1, '0110100000000111': 1, '0110100011010111': 1, '0110100010010111': 1, '0110011010110111': 1, '0110100010111011': 1, '0110011010100011': 1})
Results in precision register: {'01101000001101': 3, '01101000011101': 1, '01101000000001': 1, '01101000010011': 8, '01101000010101': 125, '01101000000111': 1, '01101000011001': 6, '01101000011011': 1, '0